This is a seq_to_1 problem (needs encoder):<br>
Process:<br>
1:Load, clean the data and tokenize<br>
2:Encode the words (Create dictionary of words)<br>
3:Word embedding<br>
4:Build rnn model (Create embdding and LSTM layers)<br>
5:Run and test

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
from keras.preprocessing.text import Tokenizer
from bs4 import BeautifulSoup
import re
import string
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


Load and clean the messages as well as encoding the lables

In [2]:

def load_clean(filepath):
    '''Load & clean the data'''

    #Load Data
    data = pd.read_csv(filepath)
    #rows_number=data.shape[0]
    messages=[]
    for message in data['v2']:
        #Extra celaning of text before Keras tokenization 
        #Removing stopwords
        nltk.download("stopwords")
        stop_words = set(stopwords.words('english'))
        message=' '.join(i for i in message.split() if i not in stop_words)
        #Here, BeauifulSoup is used to encode not completely deccoded text(decoded from html code) to html code again
        message = BeautifulSoup(message, 'lxml')
        #Later we strip away tags in the html encodings and decode them to text
        message=message.get_text()
        messages.append(message)
    
    #Encode labels
    labels=[]
    [labels.append(0) if label=="spam" else labels.append(1) for label in data['v1']]
    labels = np.asarray(labels)
    return messages,labels

 Tokenize sentences and encode their words to integers, Keras is helpful here for normalization ,tokenization as well as generating word indexes and padding the sentences

In [3]:
def encode_words(sentences):
    '''Convert words to numbers (Create dictionary of words)'''
    
    #Since we read from csv, we need to do some encoding
    #Remove u'
    sentences=[x.encode('utf-8') for x in sentences]
    #Remove \xHH characters
    sentences=[re.sub(r'[^\x00-\x7f]',r'', x) for x in sentences]
    
    #Keras tokenization (punctualtion removal, normalization and split by white space)
    tokenize = Tokenizer()
    #Fit tokenizer to the whole data
    tokenize.fit_on_texts(sentences)
    data_seq=tokenize.texts_to_sequences(sentences)
    word_index = tokenize.word_index
    #Choose the maximum number of tokens in all sequences 
    num_tokens = [len(tokens) for tokens in data_seq]
    max_seq_length=np.max(num_tokens)
    #Make sequences to have the same length(add extra zeros to the beginnings)
    data_seq = pad_sequences(data_seq, maxlen = max_seq_length,
                                padding='pre', truncating='pre')
    #print(data_seq)
    return data_seq,word_index

In [4]:
def create_model_inputs():
    '''Define model inputs'''
    
    #Resert the default graph 
    tf.reset_default_graph()
    #Model's placeholders for inputs
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs,targets,keep_prob

Create the RNN model with 2 layer LSTM

In [5]:
def build_RNN(vocabulary_size,embedding_size,inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size):
    '''Build RNN'''

    #Embedding Layer
    '''Intialize embeddings for the words. Embedding layer connects the words to the LSTM layers (words are embedded to the embedding_size vectors instead of vocabulary size vectors or one hot vectors). Here, provided by tensorflow, we used random_uniform distribution to create embeddings'''
    embedding = tf.Variable(tf.random_uniform((vocabulary_size, embedding_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)
    #Define LSTM layers
    lstms=[]
    for i in range(lstm_layer_numbers):
        lstms.append(tf.contrib.rnn.BasicLSTMCell(num_hidden))
    # Add regularization dropout to the LSTM cells
    drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) for lstm in lstms]
    # Stack up multiple LSTM layers
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(drops)
    # Getting the initial state
    initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(stacked_lstm, embed, initial_state=initial_state)
        
    return initial_state, outputs, final_state

 

In [6]:
   
def get_batches(x, y, batch_size):
    '''Using generator to return batches for train, validation and test data'''

    n_batches = len(x)//batch_size
    '''In case that the batch_size is not a multiple of data size in order to create batches with the same sizes, this line will ignore the last words in text that cannot create a full batch (although, one can consider those last words and add enough word from the beginning of the text to create a full size batch)'''
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [7]:
#Input data
emaildata_file="./spam.csv"

In [ ]:
#Load and clean data; return clean messages and labels
text_messages,labels=load_clean(emaildata_file)

In [9]:
print(labels)

[1 1 0 ..., 1 1 1]


In [10]:
#Words to int
data_sequences,word_index=encode_words(text_messages)

In [11]:
word_index

{'raining': 1592,
 'yellow': 4011,
 'four': 2311,
 'prices': 6549,
 'woods': 6739,
 "friend's": 2388,
 'hanging': 1973,
 'looking': 396,
 'electricity': 3752,
 'scold': 3754,
 'lord': 6828,
 'rp176781': 4605,
 'callin': 2480,
 'ew': 6749,
 'hearin': 8343,
 'screaming': 1608,
 'disturb': 1093,
 'prize': 107,
 'andre': 8476,
 'smsing': 7980,
 'wednesday': 1274,
 'oooh': 3205,
 'specially': 1072,
 'nigh': 7532,
 'tired': 809,
 'snuggles': 8160,
 "'wnevr": 6818,
 'second': 621,
 'attended': 7746,
 'txtno': 3131,
 'available': 616,
 'scraped': 8165,
 '2kbsubject': 4899,
 'scallies': 7419,
 'errors': 5266,
 'cooking': 2231,
 'fingers': 1223,
 'maraikara': 6845,
 'hero': 5162,
 "how've": 6751,
 'y87': 6931,
 'here': 233,
 'specialise': 5727,
 '47': 7730,
 'china': 2793,
 'dogwood': 7964,
 'dorm': 3261,
 '08718711108': 4829,
 'previews': 5968,
 '84122': 5275,
 'w111wx': 2211,
 'kids': 1035,
 '84128': 2631,
 'eastenders': 3427,
 '09058091870': 7880,
 "i'd": 854,
 "i'm": 6,
 'spotty': 6044,
 'go

In [12]:
print(data_sequences)

[[   0    0    0 ...,   20 4361   98]
 [   0    0    0 ...,  422    2 1885]
 [   0    0    0 ...,  618  343 2936]
 ..., 
 [   0    0    0 ...,   33  504 8817]
 [   0    0    0 ...,  993  151   12]
 [   0    0    0 ...,   88  436  219]]


In [13]:
#Split the data into train,test and validation sets
#First split train and test parts, then split train part to train and validation parts
X_train, X_test, y_train, y_test = train_test_split(data_sequences, labels, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [14]:
#Define Parameters
#Vocab size plus one for 0, the int number that added for padding
n_input = len(word_index)+1
# number of units
num_hidden = 256
lstm_layer_numbers=2
embed_size=300
batch_size= 250
learning_rate=0.001

Build and execute the graph

In [15]:
inputs,targets,keep_prob=create_model_inputs()
initial_state, outputs, final_state = build_RNN(n_input,embed_size,inputs,num_hidden,lstm_layer_numbers,keep_prob,batch_size)

# Loss and optimizer
#second parameter: one output which indicates if the input message is spam or ham
predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid,
                                                weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                                biases_initializer=tf.zeros_initializer())

loss_function = tf.losses.mean_squared_error(targets, predictions)
optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss_function)
correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))    


#Execute the graph
sess = tf.Session()
saver = tf.train.Saver()
init_op = tf.initialize_all_variables()
sess.run(init_op)
no_of_batches_train = int(len(X_train)/batch_size)
no_of_batches_valid = int(len(X_val)/batch_size)

epochs = 35
for epoch in range(epochs):
    state = sess.run(initial_state)
    avg_cost_train = 0 
    avg_acc_train= 0
    for ii, (x, y) in enumerate(get_batches(X_train, y_train, batch_size), 1):
        _, cost, acc= sess.run([optimizer, loss_function,accuracy], feed_dict={inputs: x,
                                                        targets: y[:, None],keep_prob: 0.5,initial_state: state})
        
        avg_cost_train += cost / no_of_batches_train
        avg_acc_train += acc / no_of_batches_train
    state_val = sess.run(initial_state)
    avg_cost_val = 0  
    avg_acc_val = 0
    for ii, (x, y) in enumerate(get_batches(X_val, y_val, batch_size), 1):
        _, cost, acc= sess.run([optimizer, loss_function, accuracy], feed_dict={inputs: x,
                                                        targets: y[:, None],keep_prob: 1,initial_state: state_val})
        
        avg_cost_val += cost / no_of_batches_valid
        avg_acc_val += acc / no_of_batches_valid
    print("Epoch:", epoch+1, "cost_train=", avg_cost_train, "cost_val=", avg_cost_val)
    print("acc_train=", avg_acc_train, "acc_val=", avg_acc_val) 
#Save the model into a file 
checkpoint="./model/savedmodel.ckpt"
save_path = saver.save(sess, checkpoint)
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch:', 1, 'cost_train=', 0.23654443770647046, 'cost_val=', 0.23248936732610065)
('acc_train=', 0.63759224329675945, 'acc_val=', 0.78982198238372803)
('Epoch:', 2, 'cost_train=', 0.23368214390107564, 'cost_val=', 0.22953258951505029)
('acc_train=', 0.65787559747695923, 'acc_val=', 0.80250777800877882)
('Epoch:', 3, 'cost_train=', 0.22993492228644236, 'cost_val=', 0.22661814590295154)
('acc_train=', 0.68526858942849289, 'acc_val=', 0.80738693475723267)
('Epoch:', 4, 'cost_train=', 0.2268960763301168, 'cost_val=', 0.22372841338316601)
('acc_train=', 0.70241533858435501, 'acc_val=', 0.80933860937754298)
('Epoch:', 5, 'cost_train=', 0.22416281593697412, 'cost_val=', 0.22087885936101276)
('acc_train=', 0.71579817363194043, 'acc_val=', 0.81324191888173436)
('Epoch:', 6, 'cost_train=', 0.22121161435331615, 'cost_val=', 0.21803827583789825)
('acc_train=', 0.73649974380220695, 'acc_val=', 0.82104857762654615)
('Epoch:'

In [16]:
#Test the saved model
no_of_batches_test = int(len(X_test)/batch_size)
sess = tf.Session()
# Load the model
saver = tf.train.Saver()
saver.restore(sess, checkpoint)
state_test = sess.run(initial_state)
avg_cost_test = 0  
avg_acc_test = 0  
for ii, (x, y) in enumerate(get_batches(X_test, y_test, batch_size), 1):
    _, cost, acc = sess.run([optimizer, loss_function, accuracy], feed_dict={inputs: x,
                                                    targets: y[:, None],keep_prob: 1,initial_state: state_test})
    avg_cost_test += cost / no_of_batches_test
    avg_acc_test += acc / no_of_batches_test
print("Test loss",avg_cost_test) 
print("Test Accuracy",avg_acc_test)
sess.close()

INFO:tensorflow:Restoring parameters from ./model/savedmodel.ckpt
('Test loss', 0.14462399482727051)
('Test Accuracy', 0.85788622498512268)
